In [1]:
#MASTER_NOTEBOOK_CLASSIFICATION

# Classification Project!

Why are our customers churning?
Some questions I have include:
*	Could the month in which they signed up influence churn? i.e. if a cohort is identified by tenure, is there a cohort or cohorts who have a higher rate of churn than other cohorts? (Plot the rate of churn on a line chart where x is the tenure and y is the rate of churn (customers churned/total customers))
*	Are there features that indicate a higher propensity to churn? like type of internet service, type of phone service, online security and backup, senior citizens, paying more than x% of customers with the same services, etc.?
*	Is there a price threshold for specific services where the likelihood of churn increases once price for those services goes past that point? If so, what is that point for what service(s)?
*	If we looked at churn rate for month-to-month customers after the 12th month and that of 1-year contract customers after the 12th month, are those rates comparable?

## Deliverables:
1.	I will also need a report (ipynb) answering the question, "Why are our customers churning?" I want to see the analysis you did to answer my questions and lead to your findings. Please clearly call out the questions and answers you are analyzing. E.g. If you find that month-to-month customers churn more, I won't be surprised, but I am not getting rid of that plan. The fact that they churn is not because they can, it's because they can and they are motivated to do so. I want some insight into why they are motivated to do so. I realize you will not be able to do a full causal experiment, but I hope to see some solid evidence of your conclusions.
2.	I will need you to deliver to me a csv with the customer_id, probability of churn, and the prediction of churn (1=churn, 0=not_churn). I would also like a single goolgle slide that illustrates how your model works, including the features being used, so that I can deliver this to the SLT when they come with questions about how these values were derived. Please make sure you include how likely your model is to give a high probability of churn when churn doesn't occur, to give a low probability of churn when churn occurs, and to accurately predict churn.
3.	Finally, our development team will need a .py file that will take in a new dataset, (in the exact same form of the one you acquired from telco_churn.customers) and perform all the transformations necessary to run the model you have developed on this new dataset to provide probabilities and predictions.


## Acquisition
1.	Use the mysql connector to query telco_churn.customers. Assign the output of that query to the dataframe df. You want to include all the fields.
2.	Write a function, peekatdata(dataframe), that takes a dataframe as input and computes and returns the following:
    *	creates dataframe object head_df (df of the first 5 rows) and prints contents to screen
    *	creates dataframe object tail_df (df of the last 5 rows) and prints contents to screen
    *	creates tuple object shape_tuple (tuple of (nrows, ncols)) and prints tuple to screen
    *	creates dataframe object describe_df (summary statistics of all numeric variables) and prints contents to screen.
    *	prints to screen the information about a DataFrame including the index dtype and column dtypes, non-null values and memory usage.

In [2]:
import pandas as pd

path = './'
df = pd.read_csv(path + "telco_churn.csv")

In [3]:
def peekatdata(df):
    print("\nRows & Columns:\n")
    print(df.shape)
    print("\nColumn Info:\n")
    print(df.info())
    print("\nFirst 5 rows:\n")
    print(df.head())
    print("\nLast 5 rows:\n")
    print(df.tail())
    print("\nMissing Values:\n")
    missing_vals = df.columns[df.isnull().any()]
    print(df.isnull().sum())
    print("\nSummary Stats:\n")
    print(df.describe())

In [4]:
peekatdata(df)


Rows & Columns:

(7043, 27)

Column Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 27 columns):
customer_id                   7043 non-null object
gender                        7043 non-null object
senior_citizen                7043 non-null int64
partner                       7043 non-null object
dependents                    7043 non-null object
tenure                        7043 non-null int64
phone_service                 7043 non-null object
multiple_lines                7043 non-null object
internet_service_type_id      7043 non-null int64
online_security               7043 non-null object
online_backup                 7043 non-null object
device_protection             7043 non-null object
tech_support                  7043 non-null object
streaming_tv                  7043 non-null object
streaming_movies              7043 non-null object
contract_type_id              7043 non-null int64
paperless_billing             7043 no

## Data Prep
1.	Write a function, df_value_counts(dataframe), that takes a dataframe as input and computes and returns the values by frequency for each variable. Use the rule of thumb for your logic on whether or not to use the bins argument. The function will use a for loop and an in statement.
for col in df.columns: n = df[col].unique().shape[0] col_bins = min(n,10) print('%s:' % col) if df[col].dtype in ['int64','float64'] and n > 10: print(df[col].value_counts(bins=col_bins, sort=False)) else: print(df[col].value_counts()) print('\n')
1.	Missing Values:
    *	Write a function, that returns a dataframe of the column name and the number of missing values and the percentage of missing values (missing records/total records) for each of the columns that have > 0 missing values.
df.isnull().sum()
    *	Document your takeaways. For each variable:
    *	should you remove the observations with a missing value for that variable?
    *	should you remove the variable altogether?
    *	is missing equivalent to 0 (or some other constant value) in the specific case of this variable?
    *	should you replace the missing values with a value it is most likely to represent (e.g. Are the missing values a result of data integrity issues and should be replaced by the most likely value?)
    *	Handle the missing values in the way you recommended above.
2.	Transform churn such that "yes" = 1 and "no" = 0
3.	Compute a new feature, tenure_year, that is a result of translating tenure from months to years.
4.	Figure out a way to capture the information contained in phone_service and multiple_lines into a single variable of dtype int. Write a function that will transform the data and place in a new column phone_id in df_sql. Be sure you have documented your function and logic well.
5.	Figure out a way to capture the information contained in dependents and partner into a single variable of dtype int. Transform the data and place in a new column household_type_id in df_sql. Be sure you have documented your function and logic well.
6.	Figure out a way to capture the information contained in streaming_tv and streaming_movies into a single variable of dtype int. Transform the data and place in a new column streaming_services in df_sql. Be sure you have documented your function and logic well.
7.	Figure out a way to capture the information contained in online_security and online_backup into a single variable of dtype int. Transform the data and place in a new column online_security_backup in df_sql. Be sure you have documented your function and logic well.
8.	Data Split
    *	Split data into train (70%) & test (30%) samples. You should end with 2 data frames: train_df and test_df
train_df, test_df = train_test_split(df_sql, test_size = .30, random_state = 123, stratify = df[['churn']])
9.	Variable Encoding
    *	write an encoder (fit and transform on train_df) for each non-numeric variable. Use that encoder object to transform on test_df
10.	Numeric Scaling
    *	Fit a min_max_scaler to train_df. Transform monthly_charges and total_charges variables in train_df using the scaler. Then use the scaler object to transform test_df.


In [5]:
def df_value_counts(df):
    for col in df.columns: 
        n = df[col].unique().shape[0] 
        col_bins = min(n,10) 
        if df[col].dtype in ['int64','float64'] and n > 10:
            print('%s:' % col)
            print(df[col].value_counts(bins=col_bins, sort=False)) 
        else: 
            print(df[col].value_counts()) 
        print('\n')

In [6]:
df_value_counts(df)

5081-NWSUP    1
9154-QDGTH    1
1895-QTKDO    1
7434-SHXLS    1
6278-FEPBZ    1
5299-SJCZT    1
1934-SJVJK    1
1728-CXQBE    1
7338-ERIVA    1
7825-GKXMW    1
6670-MFRPK    1
2256-YLYLP    1
7874-ECPQJ    1
2676-OXPPQ    1
0302-JOIVN    1
3716-UVSPD    1
6718-BDGHG    1
0141-YEAYS    1
8915-NNTRC    1
6817-WTYHE    1
7319-ZNRTR    1
8388-FYNPZ    1
3489-HHPFY    1
2516-XSJKX    1
1100-DDVRV    1
8051-HJRLT    1
2740-JFBOK    1
2947-DOMLJ    1
1134-YWTYF    1
5297-MDOIR    1
             ..
3043-TYBNO    1
1536-HBSWP    1
2933-FILNV    1
0909-SDHNU    1
2673-ZALNP    1
1915-IOFGU    1
0975-VOOVL    1
4648-YPBTM    1
3892-NXAZG    1
0567-GGCAC    1
3911-RSNHI    1
8645-KWHJO    1
1559-DTODC    1
6918-UMQCG    1
4283-FUTGF    1
3865-ZYKAD    1
5692-ICXLW    1
8050-XGRVL    1
6575-SUVOI    1
1934-MKPXS    1
8313-NDOIA    1
2458-EOMRE    1
0599-XNYDO    1
0842-IWYCP    1
1231-YNDEK    1
5310-NOOVA    1
2753-JMMCV    1
6900-RBKER    1
2568-BRGYX    1
5882-CMAZQ    1
Name: customer_id, Lengt

In [7]:
def percent_missing(df):
    missing_table = df.isnull().sum()/df.shape[0]*100
    return missing_table

In [8]:
percent_missing(df)

customer_id                   0.0
gender                        0.0
senior_citizen                0.0
partner                       0.0
dependents                    0.0
tenure                        0.0
phone_service                 0.0
multiple_lines                0.0
internet_service_type_id      0.0
online_security               0.0
online_backup                 0.0
device_protection             0.0
tech_support                  0.0
streaming_tv                  0.0
streaming_movies              0.0
contract_type_id              0.0
paperless_billing             0.0
payment_type_id               0.0
monthly_charges               0.0
total_charges                 0.0
churn                         0.0
contract_type_id.1            0.0
contract_type                 0.0
internet_service_type_id.1    0.0
internet_service_type         0.0
payment_type_id.1             0.0
payment_type                  0.0
dtype: float64

#### There is no missing data for any columns, so for now, we will not remove anything and there is no need to replace values.

In [9]:
def make_binary(df):
    df['churn'] == 'Yes'
    (df['churn'] == 'Yes').astype(int)
    df['churn'] = (df['churn'] == 'Yes').astype(int)
    return(df.head())

In [10]:
make_binary(df)

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,payment_type_id,monthly_charges,total_charges,churn,contract_type_id.1,contract_type,internet_service_type_id.1,internet_service_type,payment_type_id.1,payment_type
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,2,59.9,542.4,0,1,Month-to-month,1,DSL,2,Mailed check
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,4,69.4,571.45,0,1,Month-to-month,1,DSL,4,Credit card (automatic)
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,1,48.2,340.35,0,1,Month-to-month,1,DSL,1,Electronic check
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,1,No,...,1,25.1,25.1,1,1,Month-to-month,1,DSL,1,Electronic check
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,1,Yes,...,3,30.5,30.5,1,1,Month-to-month,1,DSL,3,Bank transfer (automatic)


In [11]:
df = df.assign(tenure_year=df.tenure/12).round(2)

In [12]:
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,monthly_charges,total_charges,churn,contract_type_id.1,contract_type,internet_service_type_id.1,internet_service_type,payment_type_id.1,payment_type,tenure_year
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,59.9,542.4,0,1,Month-to-month,1,DSL,2,Mailed check,0.75
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,69.4,571.45,0,1,Month-to-month,1,DSL,4,Credit card (automatic),0.75
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,48.2,340.35,0,1,Month-to-month,1,DSL,1,Electronic check,0.58
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,1,No,...,25.1,25.1,1,1,Month-to-month,1,DSL,1,Electronic check,0.08
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,1,Yes,...,30.5,30.5,1,1,Month-to-month,1,DSL,3,Bank transfer (automatic),0.08


# Data Prep
# Question 5
5.	Figure out a way to capture the information contained in dependents and partner into a single variable of dtype int. Transform the data and place in a new column household_type_id in df_sql. Be sure you have documented your function and logic well.

In [13]:
df.columns

Index(['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'internet_service_type_id',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_tv', 'streaming_movies', 'contract_type_id',
       'paperless_billing', 'payment_type_id', 'monthly_charges',
       'total_charges', 'churn', 'contract_type_id.1', 'contract_type',
       'internet_service_type_id.1', 'internet_service_type',
       'payment_type_id.1', 'payment_type', 'tenure_year'],
      dtype='object')

In [14]:
df[['partner','dependents']].head()

,partner,dependents
0,No,No
1,No,Yes
2,No,No
3,No,No
4,Yes,Yes


### Partner and dependents columns are strings with 'Yes' or 'No' values. Turn columns into boolean values. 

In [15]:
df['partner'] = df['partner'] == 'Yes'

In [16]:
df['dependents'] = df['dependents'] == 'Yes'

In [17]:
df.head(10)

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,monthly_charges,total_charges,churn,contract_type_id.1,contract_type,internet_service_type_id.1,internet_service_type,payment_type_id.1,payment_type,tenure_year
0,0003-MKNFE,Male,0,False,False,9,Yes,Yes,1,No,...,59.90,542.4,0,1,Month-to-month,1,DSL,2,Mailed check,0.75
1,0013-MHZWF,Female,0,False,True,9,Yes,No,1,No,...,69.40,571.45,0,1,Month-to-month,1,DSL,4,Credit card (automatic),0.75
2,0015-UOCOJ,Female,1,False,False,7,Yes,No,1,Yes,...,48.20,340.35,0,1,Month-to-month,1,DSL,1,Electronic check,0.58
3,0023-HGHWL,Male,1,False,False,1,No,No phone service,1,No,...,25.10,25.1,1,1,Month-to-month,1,DSL,1,Electronic check,0.08
4,0032-PGELS,Female,0,True,True,1,No,No phone service,1,Yes,...,30.50,30.5,1,1,Month-to-month,1,DSL,3,Bank transfer (automatic),0.08
5,0067-DKWBL,Male,1,False,False,2,Yes,No,1,Yes,...,49.25,91.1,1,1,Month-to-month,1,DSL,1,Electronic check,0.17
6,0076-LVEPS,Male,0,False,True,29,No,No phone service,1,Yes,...,45.00,1242.45,0,1,Month-to-month,1,DSL,2,Mailed check,2.42
7,0082-LDZUE,Male,0,False,False,1,Yes,No,1,No,...,44.30,44.3,0,1,Month-to-month,1,DSL,2,Mailed check,0.08
8,0096-BXERS,Female,0,True,False,6,Yes,Yes,1,No,...,50.35,314.55,0,1,Month-to-month,1,DSL,1,Electronic check,0.50
9,0096-FCPUF,Male,0,False,False,30,Yes,Yes,1,Yes,...,64.50,1888.45,0,1,Month-to-month,1,DSL,2,Mailed check,2.50


### Sum the two columns and turn this into the new column called household_type_id. This household_type will require a KEY to understand it.

In [18]:
## Household_type_id has 3 values: 
### 0 no partner & no dependents; 1 partner or dependents; 2 partner and dependentsdf['household_type_id'] = df['dependents'].astype(int) + df['partner'].astype(int)

In [19]:
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,total_charges,churn,contract_type_id.1,contract_type,internet_service_type_id.1,internet_service_type,payment_type_id.1,payment_type,tenure_year,household_type_id
0,0003-MKNFE,Male,0,False,False,9,Yes,Yes,1,No,...,542.4,0,1,Month-to-month,1,DSL,2,Mailed check,0.75,0
1,0013-MHZWF,Female,0,False,True,9,Yes,No,1,No,...,571.45,0,1,Month-to-month,1,DSL,4,Credit card (automatic),0.75,1
2,0015-UOCOJ,Female,1,False,False,7,Yes,No,1,Yes,...,340.35,0,1,Month-to-month,1,DSL,1,Electronic check,0.58,0
3,0023-HGHWL,Male,1,False,False,1,No,No phone service,1,No,...,25.1,1,1,Month-to-month,1,DSL,1,Electronic check,0.08,0
4,0032-PGELS,Female,0,True,True,1,No,No phone service,1,Yes,...,30.5,1,1,Month-to-month,1,DSL,3,Bank transfer (automatic),0.08,2


## Household_type_id has 3 values: 
### 0 no partner & no dependents; 1 partner or dependents; 2 partner and dependents

## Data Exploration
### Deliverable
I will also need a report (ipynb) answering the question, "Why are our customers churning?" I want to see the analysis you did to answer my questions and lead to your findings. Please clearly call out the questions and answers you are analyzing. E.g. If you find that month-to-month customers churn more, I won't be surprised, but I am not getting rid of that plan. The fact that they churn is not because they can, it's because they can and they are motivated to do so. I want some insight intowhy they are motivated to do so. I realize you will not be able to do a full causal experiment, but I hope to see some solid evidence of your conclusions.
1.	Could the month in which they signed up influence churn? i.e. if a cohort is identified by tenure, is there a cohort or cohorts who have a higher rate of churn than other cohorts? (Plot the rate of churn on a line chart where x is the tenure and y is the rate of churn (customers churned/total customers))
2.	Are there features that indicate a higher propensity to churn? like type of internet service, type of phone service, online security and backup, senior citizens, paying more than x% of customers with the same services, etc.?
3.	Is there a price threshold for specific services where the likelihood of churn increases once price for those services goes past that point? If so, what is that point for what service(s)?
4.	If we looked at churn rate for month-to-month customers after the 12th month and that of 1-year contract customers after the 12th month, are those rates comparable?
5.	Controlling for services (phone_id, internet_service_type_id, online_security_backup, device_protection, tech_support, and contract_type_id), is the mean monthly_charges of those who have churned significantly different from that of those who have not churned?
import scipy as sp import numpy as np sp.stats.ttest_ind(df.dropna()[train_df['churn']==1]['monthly_charges'], df.dropna()[train_df['churn']==0]['monthly_charges'])
6.	How much of monthly_charges can be explained by internet_service_type? (hint: correlation test). State your hypotheses and your conclusion clearly.
7.	How much of monthly_charges can be explained by internet_service_type + phone service type (0, 1, or multiple lines). State your hypotheses and your conclusion clearly.
8.	Create visualizations exploring the interactions of variables (independent with independent and independent with dependent). The goal is to identify features that are related to churn, identify any data integrity issues, understand 'how the data works', e.g. we may find that all who have online services also have device protection. In that case, we don't need both of those. (The visualizations done in your analysis for questions 1-5 count towards the requirements below)
•	Each independent variable (except for customer_id) must be visualized in at least two plots, and at least 1 of those compares the independent variable with the dependent variable.
•	For each plot where x and y are independent variables, add a third dimension (where possible), of churn represented by color.
•	Use subplots when plotting the same type of chart but with different variables.
•	Adjust the axes as necessary to extract information from the visualizations (adjusting the x & y limits, setting the scale where needed, etc.)
•	Add annotations to at least 5 plots with a key takeaway from that plot.
•	Use plots from matplotlib, pandas and seaborn.
•	Use each of the following:
•	sns.heatmap
•	pd.crosstab (with color)
•	pd.scatter_matrix
•	sns.barplot
•	sns.swarmplot
•	sns.pairplot
•	sns.jointplot
•	sns.relplot or plt.scatter
•	sns.distplot or plt.hist
•	sns.boxplot
•	plt.plot
•	Use at least one more type of plot that is not included in the list above.
9.	What can you say about each variable's relationship to churn, based on your initial exploration? If there appears to be some sort of interaction or correlation, assume there is no causal relationship and brainstorm (and document) ideas on reasons there could be correlation.
•	phone_id
•	internet_service_type_id
•	online_security_backup
•	device_protection
•	tech_support
•	contract_type_id
•	senior_citizen
•	tenure
•	tenure_year
•	monthly_charges
•	total_charges
•	payment_type_id
•	paperless_billing
•	contract_type_id
•	gender
10.	Summarize your conclusions, provide clear answers to the specific questions, and summarize any takeaways/action plan from the work above.